In [192]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date

In [201]:
# see the data
#types = [np.int32,np.int32,np.int32,None,np.int32,None,None]
off_train = pd.read_csv('./coupon/ccf_offline_stage1_train.csv',dtype=str)
COLUMNS = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_train.columns = COLUMNS
#print(off_train.head())

off_test = pd.read_csv('./coupon/ccf_offline_stage1_test_revised.csv',dtype=str)
off_test.columns = COLUMNS[:-1]
#print(off_test.head())

In [266]:
# datesetsplit
''' (date_received)                              
           dateset3: 20160701~20160731 (113640),features3 from 20160315~20160630  (off_test)
           dateset2: 20160515~20160615 (258446),features2 from 20160201~20160514  
           dateset1: 20160414~20160514 (138303),features1 from 20160101~20160413'''
dataset3 = off_test
feature3 = off_train[((off_train.date>='20160315')&(off_train.date<='20160630'))|((off_train.date.isnull())&(off_train.date_received>='20160315')&(off_train.date_received<='20160630'))]
dataset2 = off_train[(off_train.date_received>='20160515')&(off_train.date_received<='20160615')]
feature2 = off_train[(off_train.date>='20160201')&(off_train.date<='20160514')|((off_train.date.isnull())&(off_train.date_received>='20160201')&(off_train.date_received<='20160514'))]
dataset1 = off_train[(off_train.date_received>='20160414')&(off_train.date_received<='20160514')]
feature1 = off_train[(off_train.date>='20160101')&(off_train.date<='20160413')|((off_train.date.isnull())&(off_train.date_received>='20160101')&(off_train.date_received<='20160413'))]


In [271]:
print(dataset3.shape,dataset2.shape,dataset1.shape)
print(feature3.shape,feature2.shape,feature1.shape)

(113640, 6) (258446, 7) (137167, 7)
(1036975, 7) (812779, 7) (995240, 7)


In [272]:
############# merchant related feature   #############
"""
1.merchant related: 
      total_sales. sales_use_coupon.  total_coupon
      coupon_rate = sales_use_coupon/total_sales.  
      transfer_rate = sales_use_coupon/total_coupon. 
      merchant_avg_distance,merchant_min_distance,merchant_max_distance of those use coupon
"""
def merchants_feature_extract(set_name,data_set,feature_cols=['merchant_id','coupon_id','distance','date_received','date']):
    merchant = data_set[feature_cols].copy()
    t = merchant[['merchant_id']].copy()
    t.drop_duplicates(inplace=True)
    
    t1 = merchant[~merchant.date.isnull()][['merchant_id']].copy()
    t1['total_sales'] = 1
    t1 = t1.groupby('merchant_id').agg('sum').reset_index()
    
    t2 = merchant[(~merchant.date.isnull())&(~merchant.coupon_id.isnull())][['merchant_id']].copy()    
    t2['sales_use_coupon'] = 1   
    t2 = t2.groupby('merchant_id').agg('sum').reset_index()
    
    t3 = merchant[~merchant.coupon_id.isnull()][['merchant_id']].copy()
    t3['total_coupon'] = 1
    t3 = t3.groupby('merchant_id').agg('sum').reset_index()
    
    t4 = merchant[(~merchant.date.isnull())&(~merchant.coupon_id.isnull())][['merchant_id','distance']].copy()
    #t4.distance=np.where(~t4.distance.isnull(),t4.distance,-1)[0]
    t4.replace(np.nan,-1,inplace=True)
    t4.distance = t4.distance.astype('int')  # can be astype(int)
    t4.replace(-1,np.nan,inplace=True)
    
    def distance_feature(df,feature,new_name):
        df1 = df.groupby('merchant_id').agg(feature).reset_index()
        df1.rename(columns={'distance':new_name},inplace=True)
        return df1
    
    t5 = distance_feature(t4,'min','merchant_min_distance')
    t6 = distance_feature(t4,'max','merchant_max_distance')
    t7 = distance_feature(t4,'mean','merchant_mean_distance')
    t8 = distance_feature(t4,'median','merchant_median_distance')
    
    merchant_feature = pd.merge(t,t1,on='merchant_id',how='left')
    merchant_feature = pd.merge(merchant_feature,t2,on='merchant_id',how='left')
    merchant_feature = pd.merge(merchant_feature,t3,on='merchant_id',how='left')
    merchant_feature = pd.merge(merchant_feature,t5,on='merchant_id',how='left')
    merchant_feature = pd.merge(merchant_feature,t6,on='merchant_id',how='left')
    merchant_feature = pd.merge(merchant_feature,t7,on='merchant_id',how='left')
    merchant_feature = pd.merge(merchant_feature,t8,on='merchant_id',how='left')
    merchant_feature['sales_use_coupon'] = merchant_feature['sales_use_coupon'].replace(np.nan,0)
    merchant_feature['merchant_coupon_transfer_rate'] = merchant_feature.sales_use_coupon.astype('float').div(merchant_feature.total_coupon)
    merchant_feature['coupon_rate'] = merchant_feature.sales_use_coupon.astype('float').div(merchant_feature.total_sales)
    merchant_feature['total_coupon'] = merchant_feature['total_coupon'].fillna(0)
    merchant_feature.to_csv('./coupon/data/'+set_name,index=None)
    
## generate merchant_feature csv file
merchants_feature_extract(set_name='merchant3_feature.csv',data_set=feature3)
merchants_feature_extract(set_name='merchant2_feature.csv',data_set=feature2)
merchants_feature_extract(set_name='merchant1_feature.csv',data_set=feature1)

In [273]:
############# coupon related feature   #############
"""
2.coupon related: 
      discount_rate. discount_man. discount_jian. is_man_jian
      day_of_week,day_of_month. (date_received)
"""
def calc_discount_rate(s):
    s = s.split(':')
    if len(s) == 1:
        return np.nan
    else:
        return 1.0-float(s[1])/float(s[0])

def get_discount_man(s):
    s = s.split(':')
    if len(s) == 1:
        return np.nan
    else:
        return int(s[0])
    
def get_discount_jian(s):
    s = s.split(':')
    if len(s) == 1:
        return np.nan
    else:
        return int(s[1])
    
def is_man_jian(s): # if have discount is man_jian,then 1,else 0
    s = s.split(':')
    if len(s) == 1:
        return 0
    else:
        return 1

def coupon_related_feature(save_name,data_sets,date_start):
    data_set = data_sets.copy()
    data_set['day_of_week'] = data_set.date_received.apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:])).weekday()+1)
    data_set['day_of_month'] = data_set.date_received.apply(lambda x:int(x[6:]))
    data_set['days_distance'] = data_set.date_received.apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:]))-date_start).days)
    data_set['discount_man'] =  data_set.discount_rate.apply(get_discount_man)
    data_set['discount_jian'] = data_set.discount_rate.apply(get_discount_jian)
    data_set['is_man_jian'] = data_set.discount_rate.apply(is_man_jian)
    data_set['discount_rate'] = data_set.discount_rate.apply(calc_discount_rate)
    d = data_set[['coupon_id']].copy()
    d['coupon_count'] = 1
    d = d.groupby('coupon_id').agg('sum').reset_index()
    data_set = pd.merge(data_set,d,on='coupon_id',how='left')
    data_set.to_csv('./coupon/data/'+save_name,index=None)
    
# generate coupon_related_feature
coupon_related_feature(save_name='coupon3_feature.csv',data_sets=dataset3,date_start=date(2016,6,30))
coupon_related_feature(save_name='coupon2_feature.csv',data_sets=dataset2,date_start=date(2016,5,14))
coupon_related_feature(save_name='coupon1_feature.csv',data_sets=dataset1,date_start=date(2016,4,13))

In [274]:
############# user related feature   #############
"""
3.user related: 
      count_merchant. 
      user_avg_distance, user_min_distance,user_max_distance. 
      buy_use_coupon. buy_total. coupon_received.
      buy_use_coupon/coupon_received. 
      buy_use_coupon/buy_total
      user_date_datereceived_gap
      
"""
def get_user_date_datereceived_gap(s):
    s = s.split(':')
    return (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8])) - 
            date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days

def user_related_features(save_name,data_set,features=['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']):
    user = data_set[features].copy()
    
    t = user[['user_id']].copy()
    t.drop_duplicates(inplace=True)
    
    t1 = user[~user.date.isnull()][['user_id','merchant_id']].copy()
    t1.drop_duplicates(inplace=True) # drop the subset['user_id','merchant_id']
    #t1.merchant_id = 1
    #t1 = t1.groupby('user_id').agg('sum').reset_index()
    t1 = t1.groupby('user_id').agg('count').reset_index()
    t1.rename(columns = {'merchant_id':'count_merchant'},inplace=True)
    
    t2 = user[(~user.date.isnull())&(~user.coupon_id.isnull())][['user_id','distance']].copy()
    t2.replace(np.nan,-1,inplace=True)
    t2.distance = t2.distance.astype(dtype=int,copy=True)
    t2.replace(-1,np.nan,inplace=True)
    
    def distance_features(data,agg_fun,feature_name):
        df = data.groupby('user_id').agg(agg_fun).reset_index()
        df.rename(columns={'distance':feature_name})
        return df
    
    t3 = distance_features(data=t2,agg_fun='min',feature_name='user_min_distance')
    t4 = distance_features(t2,'max','user_max_distance')
    t5 = distance_features(t2,'mean','user_mean_distance')
    t6 = distance_features(t2,'median','user_median_distance')
    
    t7 = user[(~user.date.isnull())&(~user.coupon_id.isnull())][['user_id']].copy()
    t7['buy_use_coupon'] = 1
    t7 = t7.groupby('user_id').agg('sum').reset_index()
    
    t8 = user[~user.date.isnull()][['user_id']].copy()
    t8['buy_total'] = 1
    t8 = t8.groupby('user_id').agg('sum').reset_index()
    
    t9 = user[~user.coupon_id.isnull()][['user_id']].copy()
    t9['coupon_received'] = 1
    t9 = t9.groupby('user_id').agg('sum').reset_index()
    
    t10 = user[(~user.date_received.isnull())&(~user.date.isnull())][['user_id','date_received','date']].copy()
    t10['user_date_datereceived_gap'] = t10.date+':'+t10.date_received
    t10['user_date_datereceived_gap'] = t10['user_date_datereceived_gap'].apply(get_user_date_datereceived_gap)
    t10 = t10[['user_id','user_date_datereceived_gap']].copy()
    #print(type(t10.user_date_datereceived_gap.values[0]))
    #t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.astype(float)
    
    def data_gap_feature(data,agg_fun,feature_name):
        df = data.groupby('user_id').agg(agg_fun).reset_index()
        df.rename(columns={'user_date_datereceived_gap':feature_name},inplace=True)
        return df
    
    t11 = data_gap_feature(t10,'mean','avg_user_date_datereceived_gap')
    t12 = data_gap_feature(t10,'min','min_user_date_datereceived_gap')
    t13 = data_gap_feature(t10,'max','max_user_date_datereceived_gap')
    
    user_feature = pd.merge(t,t1,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t3,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t4,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t5,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t6,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t7,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t8,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t9,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t10,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t11,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t12,on='user_id',how='left')
    user_feature = pd.merge(user_feature,t13,on='user_id',how='left')
    user_feature.count_merchant.fillna(0,inplace=True)
    user_feature.buy_use_coupon.fillna(0,inplace=True)
    user_feature['buy_use_coupon_rate'] = user_feature.buy_use_coupon.astype(float).div(user_feature.buy_total.astype(float))
    user_feature['user_coupon_transfer_rate'] = user_feature.buy_use_coupon.astype(float).div(user_feature.coupon_received.astype(float))
    user_feature.buy_total.fillna(0,inplace=True)
    user_feature.coupon_received.fillna(0,inplace=True)
    user_feature.to_csv('./coupon/data/'+save_name,index=None)
    
# generate user feature csv file
user_related_features('user3_feature.csv',feature3)
user_related_features('user2_feature.csv',feature2)
user_related_features('user1_feature.csv',feature1)

In [275]:
##################  user_merchant related feature #########################

"""
4.user_merchant:
      times_user_buy_merchant_before. 
"""
def user_mechant_add_feature(data_set,new_feature,used_feature):
    df = data_set[['user_id','merchant_id',used_feature]].copy()
    df1 = df[~df[used_feature].isnull()][['user_id','merchant_id']].copy()
    df1[new_feature] = 1
    df1 = df1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
    df1.drop_duplicates(inplace=True)
    return df1

def user_merchant_feature(save_name,data_set):
    all_user_merchant = data_set[['user_id','merchant_id']].copy()
    all_user_merchant.drop_duplicates(inplace=True)
    
    t = user_mechant_add_feature(data_set=data_set,new_feature='user_merchant_buy_total',used_feature='date')
    
    t1 = user_mechant_add_feature(data_set=data_set,new_feature='user_merchant_received',used_feature='coupon_id')
    
    #t2 = data_set[['user_id','merchant_id','date','date_received']].copy()
    t2 = data_set[(~data_set.date.isnull())&(~data_set.date_received.isnull())][['user_id','merchant_id']].copy()
    t2['user_merchant_buy_use_coupon'] = 1
    t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
    t2.drop_duplicates(inplace=True)
    
    t3 = data_set[['user_id','merchant_id']].copy()
    t3['user_merchant_any'] = 1 #user_id and merchant_id have relationship
    t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
    t3.drop_duplicates(inplace=True)
    
    t4 = data_set[['user_id','merchant_id','date','coupon_id']].copy()
    t4 = t4[(~t4.date.isnull())&(t4.coupon_id.isnull())][['user_id','merchant_id']]
    t4['user_merchant_buy_common'] = 1 # have coupon,then buy
    t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
    t4.drop_duplicates(inplace=True)
    
    user_merchant = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
    user_merchant = pd.merge(user_merchant,t1,on=['user_id','merchant_id'],how='left')
    user_merchant = pd.merge(user_merchant,t2,on=['user_id','merchant_id'],how='left')
    user_merchant = pd.merge(user_merchant,t3,on=['user_id','merchant_id'],how='left')
    user_merchant = pd.merge(user_merchant,t4,on=['user_id','merchant_id'],how='left')
            
    user_merchant.user_merchant_buy_common.fillna(0,inplace=True)
    user_merchant.user_merchant_buy_use_coupon.fillna(0,inplace=True)
    user_merchant['user_merchant_coupon_transfer_rate'] = user_merchant.user_merchant_buy_use_coupon.astype(float).\
                                                        div(user_merchant.user_merchant_received.astype(float))
    user_merchant['user_merchant_coupon_buy_rate'] = user_merchant.user_merchant_buy_use_coupon.astype(float).\
    div(user_merchant.user_merchant_buy_total.astype(float))
    user_merchant['user_merchant_rate'] = user_merchant.user_merchant_buy_total.astype(float).\
    div(user_merchant.user_merchant_any.astype(float))
    user_merchant['user_merchant_common_buy_rate'] = user_merchant.user_merchant_buy_common.astype(float).\
    div(user_merchant.user_merchant_buy_total.astype('float'))
    user_merchant.to_csv('./coupon/data/'+save_name,index=None)
    
# generate user_merchant_feature csv file
user_merchant_feature('user_merchant3.csv',feature3)
user_merchant_feature('user_merchant2.csv',feature2)
user_merchant_feature('user_merchant1.csv',feature1)

In [276]:
############# other feature ##################3
"""
5. other feature:
      this_month_user_receive_all_coupon_count
      this_month_user_receive_same_coupon_count
      this_month_user_receive_same_coupon_lastone
      this_month_user_receive_same_coupon_firstone
      this_day_user_receive_all_coupon_count
      this_day_user_receive_same_coupon_count
      day_gap_before, day_gap_after  (receive the same coupon)
"""

def other_features(save_name, data_set):
    t = data_set[['user_id']].copy()
    t['this_month_user_receive_all_coupon_count'] = 1
    t = t.groupby('user_id').agg('sum').reset_index()
    
    t1 = data_set[['user_id','coupon_id']].copy()
    t1['this_month_user_receive_same_coupon_count'] = 1
    t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
    
    t2 = data_set[['user_id','coupon_id','date_received']].copy()
    t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
    t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
    t2 = t2[t2.receive_number>1].copy()
    t2['max_date_received'] = t2.date_received.apply(lambda s:np.max([int(d) for d in s.split(':')]))
    t2['min_date_received'] = t2.date_received.apply(lambda s:np.min([int(d) for d in s.split(':')]))
    t2 = t2[['user_id','coupon_id','min_date_received','max_date_received']].copy()
    
    t3 = data_set[['user_id','coupon_id','date_received']].copy()
    t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')
    t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received-t3.date_received.astype(int)
    t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received.astype(int) - t3.min_date_received
    def is_firstlastone(x):
        if x==0:
            return 1
        elif x>0:
            return 0
        else:
            return -1#those only receive once
        
    t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
    t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
    t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']].copy()
    
    t4 = data_set[['user_id','date_received']].copy()
    t4['this_day_user_receive_all_coupon_count'] = 1
    t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
    
    t5 = data_set[['user_id','coupon_id','date_received']].copy()
    t5['this_day_user_receive_same_coupon_count'] = 1
    t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
    
    t6 = data_set[['user_id','coupon_id','date_received']].copy()
    t6 = t6.groupby(['user_id','coupon_id'])['date_received'].apply(lambda x:':'.join(x)).reset_index()
    t6.rename(columns={'date_received':'dates'},inplace=True)
    
    def get_day_gap_before(s):
        date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(date_received[:4]),int(date_received[4:6]),int(date_received[6:]))-date(int(d[:4]),int(d[4:6]),int(d[6:]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps) == 0:
            return -1
        else:
            return min(gaps)
        
    def get_day_gap_after(s):
        date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps)==0:
            return -1
        else:
            return min(gaps)
        
    t7 = data_set[['user_id','coupon_id','date_received']].copy()
    t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
    t7['date_received_date']=t7.date_received.astype('str')+'-'+t7.dates
    t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before) # the gap time received the coupon after last one
    t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)# the gap time received the coupon before the next one
    t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']].copy()
    
    other_feature = pd.merge(t1,t,on='user_id',how='inner')
    other_feature = pd.merge(other_feature,t3,on=['user_id','coupon_id'],how='inner')
    other_feature = pd.merge(other_feature,t4,on=['user_id','date_received'],how='inner')
    other_feature = pd.merge(other_feature,t5,on=['user_id','coupon_id','date_received'],how='inner')
    other_feature = pd.merge(other_feature,t7,on=['user_id','coupon_id','date_received'],how='inner')
    other_feature.to_csv('./coupon/data/'+save_name,index=None)
    print(other_feature.shape)
    
    
# generate other features of dataset
other_features('other_feature3.csv', dataset3)
other_features('other_feature2.csv', dataset2)
other_features('other_feature1.csv', dataset1)

(116204, 11)
(262240, 11)
(139785, 11)


In [277]:
##################  generate training and testing set ################
def get_labels(s):
    s = s.split(':')
    if s[0].isnull():
        return 0
    elif (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8]))-date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days<=15:
        return 1
    else:
        return -1

file_path = './coupon/data/'
def generate_data(file_path,save_name,coupon_file,merchant_file,user_file,user_merchant_file,other_file,data_flag):
    coupon = pd.read_csv(file_path+coupon_file,dtype=str)#'coupon3_feature.csv'
    merchant = pd.read_csv(file_path+merchant_file,dtype=str)
    user = pd.read_csv(file_path+user_file,dtype=str)
    user_merchant = pd.read_csv(file_path+user_merchant_file,dtype=str)
    other_feature = pd.read_csv(file_path+other_file,dtype=str)
    dataset = pd.merge(coupon,merchant,on='merchant_id',how='left')
    dataset = pd.merge(dataset,user,on='user_id',how='left')
    dataset = pd.merge(dataset,user_merchant,on=['user_id','merchant_id'],how='left')
    dataset = pd.merge(dataset,other_feature,on=['user_id','coupon_id','date_received'],how='left')
    dataset.drop_duplicates(inplace=True)
    print(dataset.shape)

    dataset.user_merchant_buy_total.fillna(0,inplace=True)
    dataset.user_merchant_any.fillna(0,inplace=True)
    dataset.user_merchant_received.fillna(0,inplace=True)
    dataset['is_weekend'] = dataset.day_of_week.apply(lambda x: 1 if x in ('6','7') else 0)
    weekday_dummies = pd.get_dummies(dataset.day_of_week) # one_hot_encoding
    weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
    dataset = pd.concat([dataset,weekday_dummies],axis=1)

    if data_flag:
        dataset['label'] = dataset.date+':'+dataset.date_received
        dataset['label'] = dataset.label.apply(get_labels)

    dataset.drop(['merchant_id','day_of_week','coupon_count'],axis=1,inplace=True)
    dataset.to_csv(file_path+save_name,index=None)

# generate_data
generate_data('./coupon/data/','dataset3.csv','coupon3_feature.csv','merchant3_feature.csv','user3_feature.csv',
             'user_merchant3.csv','other_feature3.csv',False)
generate_data('./coupon/data/','dataset2.csv','coupon2_feature.csv','merchant2_feature.csv','user2_feature.csv',
             'user_merchant2.csv','other_feature2.csv',False)
generate_data('./coupon/data/','dataset1.csv','coupon1_feature.csv','merchant1_feature.csv','user1_feature.csv',
             'user_merchant1.csv','other_feature1.csv',False)

(129912, 53)
(270741, 54)
(141167, 54)


In [264]:
off_test.shape

(113640, 6)

In [278]:
113640-129912

-16272